# Create table with Variant

In [0]:
%sql
CREATE or REPLACE table sales(ID INT, customer VARIANT);

In [0]:
%sql
INSERT INTO sales (ID, customer) VALUES (1, PARSE_JSON('{ "name": "person1", "revenue": 5
}')), (2, PARSE_JSON('{ "name": "person2", "age": 22 }')) ;

In [0]:
%sql
select * from sales

#Reading from Json files

In [0]:
df=spark.read.json("/Volumes/workspace/default/yt_demo/*",multiLine=True)

In [0]:
df.display()

#Adding Schema

In [0]:
df=spark.read.json("/Volumes/workspace/default/yt_demo/*",multiLine=True,schema="id long, name string, address array<struct<street:string, city:string, state:string,zip string>>, version long")

In [0]:
df.display()

In [0]:
df=spark.read.json("/Volumes/workspace/default/yt_demo/*",multiLine=True,schema="id long, name string, address struct<street:string, city:string, state:string,zip string>, version long")

In [0]:
df.display()

#Handling Dynamic Schema as Variant

In [0]:
df=spark.read.format("text").load("/Volumes/workspace/default/yt_demo")

In [0]:
df.display()

In [0]:
df=spark.read.format("text").load("/Volumes/workspace/default/yt_demo",wholetext=True)

In [0]:
df.display()

In [0]:
df_var=df.selectExpr("parse_json(value) as variant_data")

In [0]:
df_var.display()

In [0]:
%sql
create or replace table sales_var(variant_data variant)

In [0]:
df_var.write.saveAsTable("sales_var",mode="append")

In [0]:
%sql
select * from sales_var

In [0]:
%sql
copy into sales_var from "/Volumes/workspace/default/yt_demo"  FILEFORMAT = JSON
 FORMAT_OPTIONS ('singleVariantColumn' = 'variant_data',"multiLine"="true")

#Manipulating Variant Data

In [0]:
var_load_df=spark.table("sales_var")

In [0]:
var_load_df.where("variant_data:version::int=1").display()

In [0]:
# Use dot notation to access nested fields
var_load_df.where("variant_data:version::int=2").selectExpr("variant_data:address[1].city").display()

#Exploding Variant

In [0]:
var_load_df.selectExpr("variant_explode(variant_data:address)").display()

In [0]:
spark.sql("""show functions like "*variant*" """).display()

In [0]:
spark.sql(""" SELECT key, value,_metadata.file_name
  FROM sales_var,
  LATERAL variant_explode_outer(variant_data) where variant_data:version::int=2""").display()

In [0]:
spark.sql(""" SELECT key, value
  FROM {df},
  LATERAL variant_explode_outer(variant_data:address) where variant_data:version::int=2""",df=var_load_df).display()

#Safe Variant Extraction using Try_variant

In [0]:
var_load_df.selectExpr("variant_data:address[1]::int").display()

In [0]:
var_load_df.selectExpr("try_variant_get(variant_data,'$.address[1]','int')").display()

In [0]:
var_load_df.where("variant_data:version::int=2").selectExpr("variant_data:address::array<struct<street:string,city:string,state:string,zip:string>>").display()

In [0]:
spark.sql("""
CREATE TABLE sales_var_shred
LIKE sales_var
TBLPROPERTIES ('delta.enableVariantShredding' = 'true')
""")

In [0]:
df_var.write.saveAsTable("sales_var_shred",mode="append")

In [0]:
spark.sql(""" select distinct variant_data:version::int from sales_var_shred """).display()

In [0]:
spark.sql(""" select distinct variant_data:version::int from sales_var """).display()

# Variant Visual for {"a":ss,"b":mm}

# Shredded Variant Visual for {
#   "a": 1,
#   "b": "hello",
#   "c": { "x": 10, "y": 20, "z": 999 },
#   "d": [1, "oops", 3],
#   "e": "unexpected"
# }
# 


### Advantages of VARIANT's Compact Data Format

- **Improved Read/Write Performance:** VARIANT encoding outperforms JSON strings for both reads and writes.
- **Efficient Storage:** Compact columnar storage reduces I/O and improves compression.
- **Schema Flexibility:** Handles changing or unknown schemas without requiring schema evolution.
- **Optimized Querying:** Shredding allows commonly used fields to be stored as separate columns, further boosting query speed.
- **Preserves Data Fidelity:** Maintains case sensitivity and NULL values from the source data.

| filename                                                                              | block | column          | codec  | type                    | encodings          | minValue                        | maxValue                        | columnStart | compressedBytes | uncompressedBytes | values | nulls |
|---------------------------------------------------------------------------------------|-------|-----------------|--------|-------------------------|--------------------|----------------------------------|----------------------------------|-------------|-----------------|-------------------|--------|-------|
| file:/tmp/data/test/part-00000-4eb3bf36-36e3-44ad-b39b-c43995b76eb8-c000.snappy.parquet | 1     | [id]            | SNAPPY | required int32 id       | [BIT_PACKED, PLAIN]| 1                                | 1                                | 4           | 29              | 27                | 1      | 0     |
| file:/tmp/data/test/part-00000-4eb3bf36-36e3-44ad-b39b-c43995b76eb8-c000.snappy.parquet | 1     | [data, value]   | SNAPPY | required binary value   | [BIT_PACKED, PLAIN]| 0x0202010006000815416C6963650C1E | 0x0202010006000815416C6963650C1E | 33          | 44              | 42                | 1      | 0     |
| file:/tmp/data/test/part-00000-4eb3bf36-36e3-44ad-b39b-c43995b76eb8-c000.snappy.parquet | 1     | [data, metadata]| SNAPPY | required binary metadata| [BIT_PACKED, PLAIN]| 0x01020004076E616D65616765       | 0x01020004076E616D65616765       | 77          | 40              | 38                | 1      | 0     |
| file:/tmp/data/test/part-00001-4eb3bf36-36e3-44ad-b39b-c43995b76eb8-c000.snappy.parquet | 1     | [id]            | SNAPPY | required int32 id       | [BIT_PACKED, PLAIN]| 2                                | 2                                | 4           | 29              | 27                | 1      | 0     |
| file:/tmp/data/test/part-00001-4eb3bf36-36e3-44ad-b39b-c43995b76eb8-c000.snappy.parquet | 1     | [data, value]   | SNAPPY | required binary value   | [BIT_PACKED, PLAIN]| 0x020201000400060D426F620C19     | 0x020201000400060D426F620C19     | 33          | 42              | 40                | 1      | 0     |
| file:/tmp/data/test/part-00001-4eb3bf36-36e3-44ad-b39b-c43995b76eb8-c000.snappy.parquet | 1     | [data, metadata]| SNAPPY | required binary metadata| [BIT_PACKED, PLAIN]| 0x01020004076E616D65616765       | 0x01020004076E616D65616765       | 75          | 40              | 38                | 1      | 0     |